In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from PIL import Image
from sklearn import svm
from tqdm import tqdm

In [6]:
num_ids = 3
num_reps = 100
num_samples = num_reps * num_ids

indTest = np.arange(0,num_samples,num_reps)
indAll = np.arange(0,num_samples)

perf_fold = np.zeros(shape=(num_reps,))

x = np.arange(0,num_ids)
trainCat = np.repeat(x,num_reps-1)

In [ ]:
num_reps = 100
num_ids = 3
num_samples = num_reps * num_ids

indTest = np.arange(0,num_samples,num_ids)
indAll = np.arange(0,num_samples)

perf_fold = np.zeros(shape=(num_ids,))

for iFold in tqdm(range(num_reps_id)):
    
    indTrain = np.setdiff1d(indAll,indTest+iFold)
    
    dataTest = act[indTest+iFold,:]
    dataTrain = act[indTrain,:]

    clf = svm.LinearSVC(dual='auto')
    clf.fit(dataTrain,trainCat)

    dec = clf.predict(dataTest)

    diff = dec - x
    perf = np.where(diff == 0)[0]
    perf = len(perf)/num_ids

    perf_fold[iFold] = perf